In [11]:
import pandas as pd

In [12]:
child_df = pd.read_csv('children_population.csv',encoding='cp949')
child_df = child_df.iloc[1:].reset_index(drop=True)
child_df = child_df.drop(columns=['2024년_계_총인구수','2024년_계_연령구간인구수'])
cols = ['2024년_계_0~9세', '2024년_계_10~19세']

for c in cols:
    child_df[c] = (
        child_df[c]
        .astype(str)
        .str.replace(",", "")
        .astype(int)            
    )
child_df['소아청소년_인구수'] = child_df['2024년_계_0~9세'] + child_df['2024년_계_10~19세']
child_df = child_df.drop(columns=['2024년_계_0~9세','2024년_계_10~19세'])
child_df['행정구역'] = child_df['행정구역'].str.replace(r"\s*\(.*\)", "", regex=True)
child_df.to_csv('children_population_cleaned.csv', index=False, encoding='cp949')
child_df

,행정구역,소아청소년_인구수
0,서울특별시,1204547
1,부산광역시,444588
2,대구광역시,354666
3,인천광역시,467772
4,광주광역시,238255
5,대전광역시,227126
6,울산광역시,182467
7,세종특별자치시,93890
8,경기도,2258295
9,강원특별자치도,212406


In [13]:
hospital_df = pd.read_excel('hospital_counts.xlsx',header = 2)
hospital_df.iloc[1:].reset_index(drop=True)
hospital_df = hospital_df.drop(0)
hospital_df = hospital_df.drop(columns=['기준분기'])
hospital_region= hospital_df.groupby('시도')['소아청소년과'].sum()
hospital_region = hospital_region.reset_index()
hospital_region= hospital_region.rename(columns={'시도': '행정구역'})
hospital_region.to_csv('hospital_counts_cleaned.csv',index=False, encoding='cp949')

/opt/homebrew/Caskroom/miniforge/base/envs/2025SDA/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:
counts_df = pd.read_csv('hospital_counts_cleaned.csv',encoding='cp949')
population_df = pd.read_csv('children_population_cleaned.csv',encoding='cp949')

In [15]:
merged_df = pd.merge(counts_df,population_df, on='행정구역', how='left')


In [16]:
merged_df['10만명당_병원수'] = merged_df['소아청소년과'] / (merged_df['소아청소년_인구수']/100000)
merged_df

,행정구역,소아청소년과,소아청소년_인구수,10만명당_병원수
0,강원특별자치도,112,212406,52.729207
1,경기도,1404,2258295,62.170797
2,경상남도,190,509149,37.317170
3,경상북도,144,356011,40.448188
4,광주광역시,82,238255,34.416906
5,대구광역시,236,354666,66.541478
6,대전광역시,112,227126,49.311836
7,부산광역시,282,444588,63.429512
8,서울특별시,456,1204547,37.856555
9,세종특별자치시,54,93890,57.514112


In [17]:
merged_df.to_csv('processed_data.csv', index=False, encoding='cp949')
